# Client Engagement Letter Draft — Hands-on Tutorial
This notebook walks through configuring the **ClientEngagementLetterDraft** step, creating sample inputs, running the automation, and reviewing the generated outputs.

In [ ]:
import sys, subprocess
print(sys.version)

def _ensure(pkg):
    try:
        __import__(pkg)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
for _pkg in ['pandas', 'openpyxl']:
    _ensure(_pkg)
from pathlib import Path
ROOT = Path.cwd()
sys.path.append(str(ROOT / 'src'))
print('Project source path added:', ROOT / 'src')


## 1. Create sample engagement data
We'll stage demo metadata, service line rates, and a template document inside `./data/Finance/Engagements` for reporting period **202501**.

In [ ]:
from pathlib import Path
import json
from amplify_automations.core.io_utils import write_excel

PERIOD = '202501'
SUPPORT_DIR = Path('./data/Finance/Engagements')
SUPPORT_DIR.mkdir(parents=True, exist_ok=True)

metadata = [
    {
        'ClientID': 'C-1001',
        'ClientName': 'Acme Holdings',
        'FiscalYear': '2025',
        'ServiceLines': ['CONSULT', 'TAX'],
    },
    {
        'ClientID': 'C-2040',
        'ClientName': 'Global Manufacturing',
        'FiscalYear': '2025',
        'ServiceLines': ['AUDIT'],
    },
]
metadata_path = SUPPORT_DIR / f'client_metadata_{PERIOD}.json'
metadata_path.write_text(json.dumps(metadata, indent=2), encoding='utf-8')

service_lines = [
    {'ServiceLineCode': 'CONSULT', 'Description': 'Consulting Services', 'Rate': 250},
    {'ServiceLineCode': 'TAX', 'Description': 'Tax Advisory', 'Rate': 180},
    {'ServiceLineCode': 'AUDIT', 'Description': 'Audit and Assurance', 'Rate': 310},
]
write_excel(
    service_lines,
    (SUPPORT_DIR / 'service_lines.xlsx').as_posix(),
    headers=['ServiceLineCode', 'Description', 'Rate'],
)

template_path = SUPPORT_DIR / 'Engagement_Letter_Template.dotx'
template_path.write_text(
    (
        'Engagement Letter for {{ClientName}}\n'
        'Services:\n{{ServiceSummary}}\n'
        'Fiscal Year FY{{FiscalYear}}\nPrepared {{GeneratedOn}}\n'
    ),
    encoding='utf-8',
)

print('Client metadata →', metadata_path)
print('Service lines workbook →', SUPPORT_DIR / 'service_lines.xlsx')
print('Template path →', template_path)


## 2. Review the staged inputs
Inspect the JSON metadata and the Excel service line reference to understand the structure expected by the automation.

In [ ]:
import json
from amplify_automations.core.io_utils import read_excel

with open(metadata_path, encoding='utf-8') as f:
    print(json.dumps(json.load(f), indent=2))

service_table = read_excel((SUPPORT_DIR / 'service_lines.xlsx').as_posix())
service_table

## 3. Configure and run the step
We provide folder mappings, parameter placeholders, and then execute the `ClientEngagementLetterDraft` step to generate draft letters.

In [ ]:
from amplify_automations.plugins.client_engagement_letter_draft import ClientEngagementLetterDraft
from amplify_automations.core.contracts import StepIO

cfg = {
    'params': {
        'client_metadata': '{support}/client_metadata_{period}.json',
        'service_lines': '{support}/service_lines.xlsx',
        'template_path': '{support}/Engagement_Letter_Template.dotx',
        'output_folder': '{support}/Drafts_{period}',
        'manifest_path': '{support}/Drafts_{period}/manifest.json',
        'notification_log': '{support}/Drafts_{period}/notifications.txt',
        'notification_recipients': ['teams://StaffAccountant'],
    }
}
folders = {'support': str(SUPPORT_DIR), 'root': './data/Finance'}
naming = {}

step = ClientEngagementLetterDraft(cfg, folders, naming, PERIOD)
io_plan: StepIO = step.plan_io()
result = step.run(io_plan)
print('Success:', result.ok)
print('Messages:', result.messages)
result.metrics

## 4. Inspect generated artifacts
The step outputs letters (as `.docx` text files for the tutorial), a JSON manifest, and a notification log summarising who to alert.

In [ ]:
from pathlib import Path
letters_dir = Path(io_plan.outputs['letters_dir'])
manifest_path = Path(io_plan.outputs['manifest'])
notification_path = Path(io_plan.outputs['notification_log'])

print('Letters directory:', letters_dir)
print('Generated files:', [p.name for p in letters_dir.glob('*.docx')])

print('Manifest preview:')
print(manifest_path.read_text(encoding='utf-8'))

print('Notification log:')
print(notification_path.read_text(encoding='utf-8'))


## 5. Next steps
- Replace the sample metadata export with your CRM/ERP client roster.
- Expand `service_lines.xlsx` to include billing terms, partners, or delivery details.
- Drop a prior year folder into the config (`prior_letters_folder`) to roll forward letters.
- Integrate the step into a full finance pipeline or schedule it inside your orchestration tooling.